In [2]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pickle
import copy


In [3]:
def load_batch(filename):
    with open(filename, 'rb') as f:
        dataset = pickle.load(f, encoding='latin1') # Nxd(3072) (Nx (32x32x3))
        X = np.transpose(dataset['data'] / 255.) # d x N
        mean_X = np.mean(X, axis=1) # mean of each row (each feature mean)
        std_X = np.std(X, axis=1)
        X = X - np.matlib.repmat(mean_X, X.shape[1], 1).T
        X = np.divide(X, np.matlib.repmat(std_X, X.shape[1], 1).T)
        
        y = np.array(dataset['labels'])
        Y = np.transpose(np.eye(X.shape[1], np.max(y) + 1)[y]) # K x N
        return X, Y, y

def load_all(validation_size):
    X_1, Y_1, y_1 = load_batch('data/data_batch_1')
    X_2, Y_2, y_2 = load_batch('data/data_batch_2')
    X_3, Y_3, y_3 = load_batch('data/data_batch_3')
    X_4, Y_4, y_4 = load_batch('data/data_batch_4')
    X_5, Y_5, y_5 = load_batch('data/data_batch_5')
    
    X = np.concatenate((X_1, X_2, X_3, X_4, X_5[:,:-validation_size]), axis=1)
    Y = np.concatenate((Y_1, Y_2, Y_3, Y_4, Y_5[:,:-validation_size]), axis=1)
    y = np.concatenate((y_1, y_2, y_3, y_4, y_5[:-validation_size]))
    
    X_valid = X_5[:,-validation_size:]
    Y_valid = Y_5[:,-validation_size:]
    y_valid = y_5[-validation_size:]
    return X, Y, y, X_valid, Y_valid, y_valid
    

In [4]:
def batch_normalize(scores, mean, variance):
    return np.dot(np.pow(np.diag(variance + 1e-9), -0.5), (s - np.array([mean]).transpose()))

In [5]:
def softmax(s):
    exponent = np.exp(s)
    return np.divide(exponent, np.sum(exponent, axis=0))

def evaluate_classifier(X, layers):
    num_layers = len(layers)
    H = []
    S = []
    S_norm = []
    
    layer_means = []
    layer_variances = []
    
    h_prev = X
    
    for i, layer in enumerate(layers):
        if i == num_layers - 1:  # If last layer
            P = softmax(np.dot(layer["W"], h_prev) + layer["b"]) # K x N
            return H, P, S, S_norm, layer_means, layer_variances
        else:
            s = np.dot(layer["W"], h_prev) + layer["b"] # m x N
#             S.append(s)
            
#             mean = np.mean(s, axis=1) # m len
#             variance = np.mean(np.square(s - np.array([mean]).transpose()), axis=1) # m
            
#             layer_means.append(mean)
#             layer_variances.append(variance)
            
#             gamma = 1
#             beta = 0
            
#             normalized = batch_normalize(scores, mean, variance)
#             S_norm.append(normalized)
            
#             transformed = np.multiply(gamma, normalized) + beta
            
            h = np.maximum(s, 0) # ReLU; m x N
            H.append(h)
            h_prev = h

In [6]:
def compute_cost(X, Y, layers, lmb):
    H, P = evaluate_classifier(X, layers)[:2]
    n = np.sum(np.multiply(Y, P), axis=0)
    cross_entropy = np.sum(-np.log(n))
    
    w_square_sum = 0
    if lmb > 0:
        for layer in layers:
            w_square_sum += np.sum(np.diag(np.dot(layer["W"].T, layer["W"])))
    return (cross_entropy / X.shape[1]) + (lmb * w_square_sum)

def compute_gradients(X, Y, layers, lmb):
    H, P, S, S_norm, layer_means, layer_variances = evaluate_classifier(X, layers)
    G = -(Y - P)
    Nb = X.shape[1] # batch size
    
    W_gradients = []
    b_gradients = []
    for i, layer in reversed(list(enumerate(layers))): # from last to first
        if i > 0:
            grad_W = np.divide(np.dot(G, H[i - 1].T), Nb) + (2 * lmb * layer["W"]) # J w.r.t W_k
            grad_b = np.divide(np.dot(G, np.ones((Nb, 1))), Nb) # J w.r.t b_k
            
            G = np.dot(layer["W"].T, G)
            G = G * (H[i - 1] > 0).astype(int) # element-wise
            
#             gamma_grad = np.dot(np.divide(np.multiply(G, S_norm), Nb), np.ones((Nb, 1)))
#             beta_grad = np.divide(np.dot(G, np.ones((Nb, 1))))
            
#             G = np.multiply(G, np.dot()
            
            W_gradients.append(grad_W)
            b_gradients.append(grad_b)
        else: # first layer
            grad_W = np.divide(np.dot(G, X.T), Nb) + (2 * lmb * layer["W"])
            grad_b = np.divide(np.dot(G, np.ones((Nb, 1))), Nb)
            W_gradients.append(grad_W)
            b_gradients.append(grad_b)
    return W_gradients, b_gradients


In [7]:
def compute_accuracy(X, y, layers):
    _, p = evaluate_classifier(X, layers)
    argmax = np.argmax(p, axis=0) # max element index of each column
    diff = argmax - y
    return (diff == 0).sum() / X.shape[1]

In [8]:
def mini_batch_GD(X, Y, GDparams, layers, lmb, validation, calculate_loss=False):
#     print("Training samples: {}".format(X.shape[1]))
#     print("Validation samples: {}".format(validation["X"].shape[1]))
#     print("Training parameters: ", GDparams)
    J_training = []
    J_validation = []
    
    eta_diff = GDparams["eta_max"] - GDparams["eta_min"]
    eta = GDparams["eta_min"]
    t = 0 # step
    l = 0 # cycle
    
    runs_in_epoch = int(X.shape[1] / GDparams["n_batch"])
    # for epoch in range(GDparams["epochs"]):
    while l < GDparams["max_cycles"]:
        for j in range(1, runs_in_epoch):
            j_start = (j - 1) * GDparams["n_batch"]
            j_end = j * GDparams["n_batch"]
            
            X_batch = X[:, j_start:j_end]
            Y_batch = Y[:, j_start:j_end]
            
            grad_W, grad_b = compute_gradients(X_batch, Y_batch, layers, lmb)
            
            for i, layer in enumerate(layers):
                layer["W"] = layer["W"] - (eta * grad_W[-1 - i])
                layer["b"] = layer["b"] - (eta * grad_b[-1 - i])
        
            if calculate_loss and t % 100 == 0:
                J_training.append(compute_cost(X, Y, layers, lmb))
                J_validation.append(compute_cost(validation["X"], validation["Y"], layers, lmb))
                print("Step {}, training loss: {}".format(t, J_training[-1]))
                
            t += 1 # next update step
            if t % (2 * GDparams["n_s"]) == 0:
                l += 1 # next cycle
                if l == GDparams["max_cycles"]:
                    break
#                 print("Entering cycle {}, t: {}, eta: {}".format(l, t, eta))
            if t <= (2*l + 1) * GDparams["n_s"]:
                eta = GDparams["eta_min"] + (eta_diff * ((t - (2 * l * GDparams["n_s"])) / GDparams["n_s"]))
            else:
                eta = GDparams["eta_max"] - (eta_diff * (t - ((2*l + 1) * GDparams["n_s"])) / GDparams["n_s"])

    if calculate_loss:
        return layers, J_training, J_validation
    else:
        return layers

In [14]:
X, Y, y = load_batch('data/data_batch_1')
X_valid, Y_valid, y_valid = load_batch('data/data_batch_2')
# X, Y, y, X_valid, Y_valid, y_valid = load_all(validation_size=1000)
X_test, Y_test, y_test = load_batch('data/test_batch')

d = X.shape[0]
N = X.shape[1]
K = Y.shape[0]

# X: d x N
# Y: K x N

# m = 50 # hidden units

# std_dev_1 = 1 / np.sqrt(d)
# std_dev_2 = 1 / np.sqrt(m)
# W_1 = std_dev * np.random.randn(m, d)
# b_1 = std_dev * np.random.randn(m, 1)

# W_2 = std_dev * np.random.randn(K, m)
# b_2 = std_dev * np.random.randn(K, 1)
def init_network(dimensions=d, num_layers=2, hidden_units=[50], classes=K):
    
    layers = [
        {
            "W": (1 / np.sqrt(dimensions)) * np.random.randn(hidden_units[0], dimensions),
            "b": np.zeros((hidden_units[0], 1)),
        }
    ]
    
    if len(hidden_units) > 1:
        for prev_i, m in enumerate(hidden_units[1:]):
            layers.append({
                "W": (1 / np.sqrt(hidden_units[prev_i])) * np.random.randn(m, hidden_units[prev_i]),
                "b": np.zeros((m, 1)),
            })
        
    layers.append({
            "W": (1 / np.sqrt(hidden_units[num_layers-2])) * np.random.randn(classes, hidden_units[num_layers-2]),
            "b": np.zeros((K, 1)),
        })
    
    return layers


lmb = 0.01  # lambda
GDparams = {
    "n_batch": 100,
    "eta_min": 1e-5,
    "eta_max": 1e-1,
    "n_s": 500,
    "epochs": 10,
    "max_cycles": 1,
}
#     "n_s": 2 * np.floor(X.shape[1] / 100),
validation = {
    "X": X_valid,
    "Y": Y_valid,
}

print(GDparams)

{'n_batch': 100, 'eta_min': 1e-05, 'eta_max': 0.1, 'n_s': 500, 'epochs': 10, 'max_cycles': 1}


In [45]:
def compute_gradients_num(X, Y, layers, lmb, h):
    
    grad_W = [np.zeros(layer["W"].shape) for layer in layers]
    grad_b = [np.zeros(layer["W"].shape[0]) for layer in layers]
    c = compute_cost(X, Y, layers, lmb)
    
    for l, layer in enumerate(layers):
        for i in range(len(layer["b"])):
            layers_try = copy.deepcopy(layers)
            layers_try[l]["b"][i] = layers_try[l]["b"][i] + h
            c2 = compute_cost(X, Y, layers_try, lmb)
            grad_b[l][i] = (c2 - c) / h

        W_shape = layer["W"].shape
        for i in range(W_shape[0]):
            for j in range(W_shape[1]):
                layers_try = copy.deepcopy(layers)
                layers_try[l]["W"][i,j] = layers_try[l]["W"][i,j] + h
                c2 = compute_cost(X, Y, layers_try, lmb)
                grad_W[l][i,j] = (c2 - c) / h
        
    return grad_W, grad_b

def compute_gradients_num_slow(X, Y, layers, lmb, h):
    
    grad_W = [np.zeros(layer["W"].shape) for layer in layers]
    grad_b = [np.zeros(layer["b"].shape[0]) for layer in layers]

    for l, layer in enumerate(layers):
        for i in range(len(layer["b"])):
            layers_try1 = copy.deepcopy(layers)
            layers_try2 = copy.deepcopy(layers)                                       
                                       
            layers_try1[l]["b"][i] = layers_try1[l]["b"][i] - h
            c1 = compute_cost(X, Y, layers_try1, lmb)
            
            
            layers_try2[l]["b"][i] = layers_try2[l]["b"][i] + h
            c2 = compute_cost(X, Y, layers_try2, lmb)
            
            grad_b[l][i] = (c2 - c1) / (2*h)
        
        W_shape = layer["W"].shape 
        for i in range(W_shape[0]):
            for j in range(W_shape[1]):
                layers_try1 = copy.deepcopy(layers)
                layers_try2 = copy.deepcopy(layers) 

                layers_try1[l]["W"][i,j] = layers_try1[l]["W"][i,j] - h
                c1 = compute_cost(X, Y, layers_try1, lmb)
                
                layers_try2[l]["W"][i,j] = layers_try2[l]["W"][i,j] + h
                c2 = compute_cost(X, Y, layers_try2, lmb)
                grad_W[l][i,j] = (c2 - c1) / (2*h)
        
    return grad_W, grad_b

In [64]:
feature_dims = 10
samples = 1

layers_g = init_network(dimensions=feature_dims, num_layers=4, hidden_units=[20,20,20], classes=K)
X_g = X[0:feature_dims, 0:samples] #d X N
Y_g = Y[:, 0:samples] #K x N

grad_W, grad_b = compute_gradients(X_g, Y_g, layers_g, lmb=0)
ngrad_w, ngrad_b = compute_gradients_num_slow(X_g, Y_g, layers_g, lmb=0, h=1e-6)
print("\ngrad b (last to first):")

for i in range(len(grad_b)):
    diffs = np.abs(grad_b[len(grad_b) - 1 - i].T - ngrad_b[i]) / np.maximum(1e-6, np.abs(grad_b[len(grad_b) - 1 - i].T) + np.abs(ngrad_b[i]))
    print(diffs)
    print("Layer {} mean diff: {}".format(len(grad_b) - i, np.mean(diffs)))


print("\ngrad W (last to first):")
for i in range(len(grad_W)):
    diffs = np.abs(grad_W[len(grad_W) - 1 - i] - ngrad_w[i]) / np.maximum(1e-6, np.abs(grad_W[len(grad_W) - 1 - i]) + np.abs(ngrad_w[i]))
    print(diffs)
    print("Layer {} mean diff: {}".format(len(grad_W) - i, np.mean(diffs)))



grad b (last to first):
[[0.00000000e+00 4.94719453e-10 0.00000000e+00 2.23470375e-09
  0.00000000e+00 4.49989568e-10 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.34395266e-10 1.39301063e-10
  3.57721164e-10 8.29952460e-10 6.77770968e-11 0.00000000e+00
  1.51499017e-10 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
Layer 4 mean diff: 2.4800294187506034e-10
[[0.00000000e+00 6.77545235e-10 0.00000000e+00 0.00000000e+00
  1.19014249e-09 1.01316313e-10 2.06984228e-10 0.00000000e+00
  1.75638534e-10 0.00000000e+00 4.34733307e-10 1.94624255e-08
  0.00000000e+00 0.00000000e+00 0.00000000e+00 9.24209451e-11
  0.00000000e+00 1.00468838e-10 0.00000000e+00 0.00000000e+00]]
Layer 3 mean diff: 1.1220837718285377e-09
[[1.91256076e-10 2.08990307e-10 1.61221247e-10 2.40845168e-10
  1.80296044e-10 0.00000000e+00 3.25959237e-10 7.38770597e-11
  0.00000000e+00 7.18533570e-11 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.